In [3]:
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split as tt

In [5]:
imdb=pd.read_csv('IMDB Dataset.csv')

In [7]:
def clean_review(text):
    if not isinstance(text, str):
        return ""

    # Remove HTML tags
    text = re.sub(r'<.*?>', ' ', text)

    # Convert to lowercase
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text


In [9]:
imdb['review'] = imdb['review'].apply(clean_review)

In [11]:
vectorizer=TfidfVectorizer(max_features=5000)

In [13]:
review_vector=vectorizer.fit_transform(imdb['review'])

In [15]:
re_vec=review_vector.toarray()

In [17]:
imdb['review_vector']=list(re_vec)

In [19]:
le=LabelEncoder()
imdb['sentiment']=le.fit_transform(imdb['sentiment'])

In [21]:
x=imdb['review_vector']
y=imdb['sentiment']

In [23]:
x_train,x_test,y_train,y_test=tt(x,y,test_size=0.2,random_state=42)

In [25]:
x_train_matrix = np.vstack(x_train.values)
x_test_matrix = np.vstack(x_test.values)
y_train_matrix=np.vstack(y_train.values)
y_test_matrix=np.vstack(y_test.values)

In [28]:
!pip install xgboost

In [32]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators': [100, 200],
    'max_depth': [4, 6, 8],
    'learning_rate': [0.05, 0.1, 0.2],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1],
    'gamma': [0, 1],
}

xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
grid = GridSearchCV(xgb, param_grid=params, scoring='accuracy', cv=3, verbose=1, n_jobs=-1)
grid.fit(x_train_matrix, y_train_matrix.ravel())

print("✅ Best XGBoost Accuracy:", grid.best_score_)
print("🔧 Best Parameters:", grid.best_params_)


Fitting 3 folds for each of 144 candidates, totalling 432 fits


OSError: [WinError 1450] Insufficient system resources exist to complete the requested service

In [54]:
params = {
    'max_depth': 8,                # Controls tree depth (6–8 is usually good)
    'learning_rate': 0.1,         # Step size shrinkage
    'n_estimators': 600,          # Total number of trees
    'subsample': 0.8,             # Randomly sample training data (rows)
    'colsample_bytree': 0.8,      # Randomly sample features (columns)
    'gamma': 0.1,                 # Minimum loss reduction to make a split
    'reg_alpha': 0.1,             # L1 regularization (Lasso)
    'reg_lambda': 1.0,            # L2 regularization (Ridge)
    'objective': 'binary:logistic', # For binary classification
    'eval_metric': 'logloss',     # Suitable for classification
    'use_label_encoder': False ,   # Prevents warning
    'tree_method': 'hist'
}


In [46]:
x_train_matrix = x_train_matrix.astype('float32')
x_test_matrix = x_test_matrix.astype('float32')


In [56]:
xgb_model = XGBClassifier(**params)
xgb_model.fit(x_train_matrix, y_train_matrix.ravel())

# Predict
y_pred = xgb_model.predict(x_test_matrix)

# Evaluation
print("✅ XGBoost Accuracy:", accuracy_score(y_test_matrix, y_pred))
print(classification_report(y_test_matrix, y_pred))

C:\Users\Dell\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [08:59:34] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


✅ XGBoost Accuracy: 0.8836
              precision    recall  f1-score   support

           0       0.89      0.87      0.88      4961
           1       0.88      0.89      0.89      5039

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l2'],
    'solver': ['liblinear', 'saga'],
    'max_iter': [500, 1000, 2000],
    'class_weight': [None, 'balanced']
}

# Initialize the model
log_reg = LogisticRegression()

# Grid search
grid_search = GridSearchCV(log_reg, param_grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=1)
grid_search.fit(x_train_matrix, y_train_matrix)

# Best results
print("✅ Best Accuracy:", grid_search.best_score_)
print("🔧 Best Parameters:", grid_search.best_params_)


Fitting 3 folds for each of 60 candidates, totalling 180 fits
